In [34]:
day = 6
print(f"""
    Advent of Code 2024, Day {day}     {"🎄"*day}
    Python v3.12.3
    Rico van Midde
""")


    Advent of Code 2024, Day 6     🎄🎄🎄🎄🎄🎄
    Python v3.12.3
    Rico van Midde



In [35]:
import numpy as np
from copy import deepcopy
import regex as re
from time import time

In [36]:
testdata="""
....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
"""[1:-1]

In [37]:
### Read Data ###
tic = time()

TEST = 0
if TEST:
    f = testdata.splitlines()
else:
    infile = f'../Data/day{day:02}'
    with open(infile, 'r') as f:
        f = f.readlines()

data = []
for line in f:
    line = line.strip()
    # line = list(line)
    data.append(line)

# # split lines in chunks
# data = [re.split(r"-|,", line) for line in data]
# # to numpy 2D array (characterwise)
data = np.array([list(line) for line in data])
# # to numpy 2D array
# data = np.array(data)
# data = data.astype(int)


print("data:", data, "\ntime:", time()-tic)

data: [['.' '.' '.' ... '#' '.' '.']
 ['.' '.' '.' ... '.' '.' '.']
 ['.' '.' '.' ... '.' '.' '.']
 ...
 ['.' '.' '.' ... '.' '.' '.']
 ['.' '.' '.' ... '.' '.' '.']
 ['.' '.' '.' ... '.' '#' '.']] 
time: 0.0021927356719970703


In [ ]:
### Helpers ###
def inBoundary(x, shape):
    return 0 <= x[0] < shape[0] and 0 <= x[1] < shape[1]

direction_map = {
    "N" : (-1,0),
    "E" : (0,1),
    "S" : (1,0),
    "W" : (0,-1)
}

turn = {
    "N" : "E",
    "E" : "S",
    "S" : "W",
    "W" : "N"
}

In [39]:
### part 1 ###
tic = time()
result = 0

x = np.where(data=='^') # [vertical 6, horizontal 4]
x = int(x[0][0]), int(x[1][0])
visited = set()


d = "N"
while inBoundary(x, data.shape):
    # keep track of visited
    if x not in visited:
        visited.add(x)

    # calculate step
    dx = direction_map[d]
    new_x = x[0] + dx[0], x[1] + dx[1]

    if not inBoundary(new_x, data.shape):
        break

    # turn right
    if data[new_x] == '#':
        d = turn[d]
        continue
    else:
        x = new_x

result = len(visited)

print("result:", result, '\ntime:', time()-tic)

result: 5453 
time: 0.006443023681640625


In [41]:
### Part 2 : Solution ###
tic = time()
result = 0

def recursive_walk(x, direction, visited, x_visited, depth, new_block = None):
    positions = set()
    shape = data.shape
    while True:
        # keep track of visited
        visited.add((x, direction))
        x_visited.add(x)

        # calculate next step
        dx = direction_map[direction]
        new_x = x[0] + dx[0], x[1] + dx[1]

        # boundary conditions
        if not inBoundary(new_x, shape):
            return len(positions) # = 0 for depth 1

        # turn right 
        if data[new_x] == '#' or (depth == 1 and new_x==new_block):
            direction = turn[direction]
            continue
        else:
            # found a loop in depth 1
            if depth == 1 and (new_x, direction) in visited:
                return new_block
            # create a hypothetical object in depth 0 and call the recursion (you cannot create a object on a visited square)
            elif depth == 0 and new_x not in x_visited:
                if new_x not in positions:
                    new_block = recursive_walk(x, turn[direction], deepcopy(visited), deepcopy(x_visited), depth = 1, new_block=new_x)
                    if new_block:
                        positions.add(new_block)
            x = new_x

x_start = np.where(data=='^') # [vertical 6, horizontal 4]
x_start = int(x_start[0][0]), int(x_start[1][0])

result = recursive_walk(
    x = x_start, # [vertical 6, horizontal 4]
    visited = set(),
    x_visited=set(),
    direction = "N",
    depth = 0
)

print("result:", result, '\ntime:', time()-tic)

result: 2188 
time: 38.48499536514282
